In [61]:
# imports
import pandas
import pymysql
import requests

# database connection
db = pymysql.connect("localhost","mygene","","ensembl2entrez")
cursor = db.cursor()

## Counting gene IDs
Table **entrez** and **ensembl** all IDs provided by NCBI and Ensembl, respectively from file ``gene_info.gz`` and ``gene_ensembl__gene__main.txt``

In [62]:
# querying entrez & ensembl
cursor.execute("select count(*) from entrez")
entrez_count = cursor.fetchone()[0]
cursor.execute("select count(*) from ensembl")
ensembl_count = cursor.fetchone()[0]

**Entrez** has ``{{"{:,}".format(entrez_count)}}`` gene IDs, and **Ensembl** has ``{{"{:,}".format(ensembl_count)}}`` gene IDs

## Counting mappings
Mappings have been integrated in MySQL tables **ens2ent** for mapping provided by Ensembl, and **ent2ens** for mapping provided by Entrez (see `columbo.py` script). They contain unique tuple(ncbi,ensembl):

In [63]:
# table definitions
cursor.execute("show create table ens2ent")
res = cursor.fetchone()
print(res[1])
print("\n")
cursor.execute("show create table ent2ens")
res = cursor.fetchone()
print(res[1])

CREATE TABLE `ens2ent` (
  `ensembl` varchar(255) NOT NULL,
  `symbol` varchar(255) DEFAULT NULL,
  `ncbi` int(11) NOT NULL,
  UNIQUE KEY `uniq_ensembl` (`ensembl`,`ncbi`),
  KEY `ensembl_idx` (`ensembl`),
  KEY `symbol_idx` (`symbol`),
  KEY `ncbi_idx` (`ncbi`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1


CREATE TABLE `ent2ens` (
  `ncbi` int(11) NOT NULL,
  `symbol` varchar(255) DEFAULT NULL,
  `ensembl` varchar(255) NOT NULL,
  UNIQUE KEY `uniq_entrez` (`ncbi`,`ensembl`),
  KEY `ensembl_idx` (`ensembl`),
  KEY `symbol_idx` (`symbol`),
  KEY `ncbi_idx` (`ncbi`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1


### Mappings provided by Ensembl

In [65]:
# file gene_ensembl__xref_entrezgene__dm.txt, unique ensembl/ncbi tuples
cursor.execute("select count(*) from ens2ent")
ens2ent_count = cursor.fetchone()[0]
cursor.execute("""
    select m.*
    from ensembl e 
    right join ens2ent m on e.ensembl = m.ensembl 
    where e.ensembl is null
""")
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])
ens2ent_mapping_not_in_main = pandas.DataFrame.from_records(res,columns=colnames)

**Ensembl** provides ``{{"{:,}".format(ens2ent_count)}}`` mappings. There are ``{{len(ens2ent_mapping_not_in_main)}}`` Ensembl IDs found in mappings but not in the main file.



{{ens2ent_mapping_not_in_main}}

### Mappings provided by Entrez

In [66]:
# file gene2ensembl.gz, unique ncbi/ensembl tuples
cursor.execute("select count(*) from ent2ens")
ent2ens_count = cursor.fetchone()[0]
cursor.execute("""
    select m.*
    from entrez e 
    right join ent2ens m on e.ncbi = m.ncbi 
    where e.ncbi is null
""")
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])
ent2ens_mapping_not_in_main = pandas.DataFrame.from_records(res,columns=colnames)

**Entrez** provides ``{{"{:,}".format(ent2ens_count)}}`` mappings. There are ``{{len(ent2ens_mapping_not_in_main)}}`` Entrez IDs found in mappings but not in the main file.

{{ent2ens_mapping_not_in_main}}

## When NCBI and Ensembl both agree (according to IDs)

In [67]:

cursor.execute("""
    select count(*)
    from ent2ens
    inner join ens2ent
    on ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl;
    """)
res = cursor.fetchone()
total_mappings = res[0]

cursor.execute("""
    select count(distinct(ent2ens.ncbi))
    from ent2ens
    inner join ens2ent
    where ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl;
    """)
res = cursor.fetchone()
unique_matching_ncbi_ids = res[0]

cursor.execute("""
    select sum(c)
    from (
        select count(distinct(ent2ens.ncbi)) as c
        from ent2ens
        inner join ens2ent
        on ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl
        group by ent2ens.ncbi
        having count(*) > 1
        order by ent2ens.ncbi)
    t
    """)
res = cursor.fetchone()
one_ncbi_to_many_ensembl = res[0]

cursor.execute("""
    select count(distinct(ens2ent.ensembl))
    from ent2ens
    inner join ens2ent
    where ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl;
    """)
res = cursor.fetchone()
unique_matching_ensembl_ids = res[0]

cursor.execute("""
    select sum(c)
    from (
        select count(distinct(ent2ens.ensembl)) as c
        from ens2ent
        inner join ent2ens
        on ens2ent.ensembl = ent2ens.ensembl and ens2ent.ncbi = ent2ens.ncbi
        group by ens2ent.ensembl
        having count(*) > 1
        order by ens2ent.ensembl)
    t;
    """)
res = cursor.fetchone()
one_ensembl_to_many_ncbi = res[0]

- Total mappings with **full match** between Entrez and Ensembl: ``{{"{:,}".format(total_mappings)}}``
- Amongst those full matches, ``{{"{:,}".format(unique_matching_ncbi_ids)}}`` have **unique NCBI IDs**
- ``{{"{:,}".format(one_ncbi_to_many_ensembl)}}`` are **1-NCBI-to-many-Ensembl** mappings
- And amongst those full matches, ``{{"{:,}".format(unique_matching_ensembl_ids)}}`` have **unique Ensembl IDs**
- ``{{"{:,}".format(one_ensembl_to_many_ncbi)}}`` are **1-Ensembl-to-many-NCBI** mappings


### Unique mapping example (1-NCBI-to-1-Ensembl)
First 50 unique mappings

In [68]:
cursor.execute("""
    select entrez.ncbi, entrez.ensembl, count(*)
    from ent2ens entrez
    inner join ens2ent ensembl
    on entrez.ncbi = ensembl.ncbi and entrez.ensembl = ensembl.ensembl
    group by entrez.ncbi, entrez.ensembl
    having count(*) = 1
    limit 50""")
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])

{{pandas.DataFrame.from_records(res,columns=colnames)}}

### Multiple mappings 1-NCBI-to-many-Ensembl

**Enumerate**

In [69]:
cursor.execute("""
    select entrez.ncbi,count(*)
    from ent2ens entrez
    inner join ens2ent ensembl
    on entrez.ncbi = ensembl.ncbi and entrez.ensembl = ensembl.ensembl
    group by entrez.ncbi
    having count(*) > 1
    order by count(*) desc;
    """)
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])

{{pandas.DataFrame.from_records(res,columns=colnames)}}

**Some multiple mapping examples: 1-NCBI-to-many(3)-Ensembl**

In [70]:
cursor.execute("""
    select ncbi,ensembl
    from ent2ens
    where ncbi in (
        select entrez.ncbi
        from ent2ens entrez
        inner join ens2ent ensembl
        on entrez.ncbi = ensembl.ncbi and entrez.ensembl = ensembl.ensembl
        group by entrez.ncbi
        having count(*) = 3)
    order by ncbi
    """)
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])

{{pandas.DataFrame.from_records(res,columns=colnames)}}

### Multiple mappings 1-Ensembl-to-many-NCBI

**Enumerate**

In [71]:
cursor.execute("""
    select entrez.ensembl,count(*)
    from ent2ens entrez
    inner join ens2ent ensembl
    on entrez.ncbi = ensembl.ncbi and entrez.ensembl = ensembl.ensembl
    group by entrez.ensembl
    having count(*) > 1
    order by count(*) desc;
    """)
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])

{{pandas.DataFrame.from_records(res,columns=colnames)}}

**Some multiple mapping examples: 1-Ensembl-to-many(3)-NCBI**

In [72]:
cursor.execute("""
    select ensembl,ncbi
    from ens2ent
    where ensembl in (
        select entrez.ensembl
        from ent2ens entrez
        inner join ens2ent ensembl
        on entrez.ncbi = ensembl.ncbi and entrez.ensembl = ensembl.ensembl
        group by entrez.ensembl
        having count(*) = 3)
    order by ensembl
    """)
res = list(cursor.fetchall())
colnames = tuple([desc[0] for desc in cursor.description])

{{pandas.DataFrame.from_records(res,columns=colnames)}}

## When NCBI and Ensembl both disagree (according to IDs)

In [73]:
cursor.execute("""
    select count(*)
    from ent2ens
    left join ens2ent
    on ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl
    where ens2ent.ncbi is null and ens2ent.ensembl is null
    """)
ncbi_mappings_only = cursor.fetchone()[0]
cursor.execute("""
    select count(*)
    from ens2ent
    left join ent2ens
    on ens2ent.ncbi = ent2ens.ncbi and ens2ent.ensembl = ent2ens.ensembl
    where ent2ens.ncbi is null and ent2ens.ensembl is null
    """)
ensembl_mappings_only = cursor.fetchone()[0]

- ``{{"{:,}".format(ncbi_mappings_only)}}`` mappings are only found in Entrez mapping file
- ``{{"{:,}".format(ensembl_mappings_only)}}`` mappings are only found in Ensembl mapping file

Overall, that's ``{{"{:,}".format(ncbi_mappings_only+ensembl_mappings_only)}}`` ambiguous mappings...

In [74]:
cursor.execute("""
    select entrez_ncbi, entrez_ensembl, entrez_symbol,
           ens2ent.symbol as ensembl_symbol,
           ens2ent.ensembl as ensembl_ensembl,
           ens2ent.ncbi as ensembl_ncbi
    from (
        select ent2ens.ncbi as entrez_ncbi,
               ent2ens.ensembl as entrez_ensembl,
               ent2ens.symbol as entrez_symbol
        from ent2ens
        left join ens2ent
        on ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl
        where ens2ent.ncbi is null and ens2ent.ensembl is null)
    t
    inner join ens2ent on entrez_ncbi = ens2ent.ncbi
    """)
res = list(cursor.fetchall())
unique_ambiguous_entrez_mappings = len(res)
colnames = tuple([desc[0] for desc in cursor.description])
pd_ambiguous_entrez = pandas.DataFrame.from_records(res,columns=colnames)

In [75]:
cursor.execute("""
    select entrez_ncbi, entrez_ensembl, entrez_symbol,
           ens2ent.symbol as ensembl_symbol,
           ens2ent.ensembl as ensembl_ensembl,
           ens2ent.ncbi as ensembl_ncbi
    from (
        select ent2ens.symbol as entrez_symbol,
               ent2ens.ncbi as entrez_ncbi,
               ent2ens.ensembl as entrez_ensembl
        from ent2ens
        left join ens2ent
        on ent2ens.ncbi = ens2ent.ncbi and ent2ens.ensembl = ens2ent.ensembl
        where ens2ent.ncbi is null and ens2ent.ensembl is null)
    t
    inner join ens2ent on entrez_ncbi = ens2ent.ncbi and entrez_symbol = ens2ent.symbol
    """)
res = list(cursor.fetchall())
unique_nonambiguous_entrez_mappings = len(res)
colnames = tuple([desc[0] for desc in cursor.description])
pd_disambiguated_entrez = pandas.DataFrame.from_records(res,columns=colnames)

In [76]:
cursor.execute("""
    select ensembl_ensembl,ensembl_ncbi,ensembl_symbol,
           ent2ens.symbol as entrez_symbol,
           ent2ens.ncbi as entrez_ncbi,
           ent2ens.ensembl as entrez_ensembl
    from (
        select ens2ent.symbol as ensembl_symbol,
               ens2ent.ensembl as ensembl_ensembl,
               ens2ent.ncbi as ensembl_ncbi
        from ens2ent
        left join ent2ens
        on ens2ent.ncbi = ent2ens.ncbi and ens2ent.ensembl = ent2ens.ensembl
        where ent2ens.ncbi is null and ent2ens.ensembl is null)
    t
    inner join ent2ens
    on ensembl_ensembl = ent2ens.ensembl
    """)
res = list(cursor.fetchall())
unique_ambiguous_ensembl_mappings = len(res)
colnames = tuple([desc[0] for desc in cursor.description])
pd_ambiguous_ensembl = pandas.DataFrame.from_records(res,columns=colnames)

In [77]:
cursor.execute("""
    select ensembl_ensembl,ensembl_ncbi,ensembl_symbol,
           ent2ens.symbol as entrez_symbol,
           ent2ens.ncbi as entrez_ncbi,
           ent2ens.ensembl as entrez_ensembl
    from (
        select ens2ent.symbol as ensembl_symbol,
               ens2ent.ensembl as ensembl_ensembl,
               ens2ent.ncbi as ensembl_ncbi
        from ens2ent
        left join ent2ens
        on ens2ent.ncbi = ent2ens.ncbi and ens2ent.ensembl = ent2ens.ensembl
        where ent2ens.ncbi is null and ent2ens.ensembl is null)
    t
    inner join ent2ens
    on ensembl_ensembl = ent2ens.ensembl and ensembl_symbol = ent2ens.symbol
    """)
res = list(cursor.fetchall())
unique_nonambiguous_ensembl_mappings = len(res)
colnames = tuple([desc[0] for desc in cursor.description])
pd_disambiguated_ensembl = pandas.DataFrame.from_records(res,columns=colnames)

### Unique mappings from Ensembl: what does NCBI say ?
Following are mappings (ncbi->ensembl) only found from Ensembl (total: ``{{"{:,}".format(ensembl_mappings_only)}}``), in relation to Entrez mappings based in NCBI ID. In other words, for each unique Ensembl mappings, is there another mapping in Entrez based on NCBI ID (which would obviously have a different Ensembl ID)? What are the ambiguous Ensembl mappings where Entrez says something different?

In the following table, each row shares the same Entrez ID, one from NCBI (most left), one from Ensembl (most right). Based on mappings only found on Entrez, it shows the different Ensembl ID claimed by each side.
Out of ``{{"{:,}".format(ncbi_mappings_only)}}`` unique Entrez mappings, only ``{{"{:,}".format(unique_ambiguous_entrez_mappings)}}`` can be ambiguously mapped to Ensembl (same Entrez ID, but different Ensembl ID). There are thus ``{{"{:,}".format(ncbi_mappings_only-unique_ambiguous_entrez_mappings)}}`` mappings claimed **only** by Ensembl.

{{pd_ambiguous_entrez}}

If we use symbol as a join criteria, we can disambiguate ``{{"{:,}".format(len(pd_disambiguated_entrez))}}`` mappings. The Ensembl mapping is the one kept in mygene (``ensembl_ncbi -> ensembl_ensembl`` right columns)
{{pd_disambiguated_entrez}}

### Unique mappings from Entrez: what does Ensembl say ?
Following are mappings (ncbi->ensembl) only found from Entrez (total: ``{{"{:,}".format(ncbi_mappings_only)}}``), in relation to Ensembl mappings based in NCBI ID. In other words, for each unique Entrez mappings, is there another mapping in Ensembl based on NCBI ID (which would obviously have a different Ensembl ID)? What are the ambiguous Entrez mappings where Ensembl says something different?

In the following table, each row shares the Ensembl ID, one from Ensembl (most left), one from Entrez (most right). Based on mappings only found on Ensembl, it shows the different Entrez ID claimed by each side.
Out of ``{{"{:,}".format(ensembl_mappings_only)}}`` unique Ensembl mappings, only ``{{"{:,}".format(unique_ambiguous_ensembl_mappings)}}`` can be ambiguously mapped to Entrez (same Ensembl ID, but different Entrez ID). There are thus ``{{"{:,}".format(ensembl_mappings_only-unique_ambiguous_ensembl_mappings)}}`` mappings claimed only by Ensembl.

{{pd_ambiguous_ensembl}}

If we use symbol as a join criteria, we can disambiguate ``{{"{:,}".format(len(pd_disambiguated_ensembl))}}`` mappings. The Entrez mapping is the one kept in mygene (``entrez_ncbi -> entrez_ensembl`` right columns)
{{pd_disambiguated_ensembl}}